In [1]:
import tensorflow as tf

In [2]:
!nvidia-smi

Fri Aug  5 20:33:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   43C    P0    86W / 260W |    423MiB / 11019MiB |     29%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import os
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats
import librosa
import librosa.display
import math
import sys
#sys.path.insert(0,'/home/ikwak2/hmd/notebooks')
# sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/lucashnegri-peakutils-51a679cd8428')
# sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python')
sys.path.insert(0,'utils')
from helper_code import *
from get_feature import *
from models import *
from Generator0 import *
from keras.preprocessing import sequence
# import peakutils
from scipy import special
import scipy.io as sio
from keras.preprocessing.sequence import pad_sequences
sys.path.insert(0,'/home/jk21/Documents/hmd/jk_classifier/lucashnegri-peakutils-51a679cd8428')
import peakutils

2022-08-05 20:33:42.038667: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 20:33:42.042522: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 20:33:42.042846: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-05 20:33:42.043307: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [4]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [5]:
root_dir = 'physionet.org/files/circor-heart-sound/1.0.3'
# training_data_file = root_dir + '/' + 'training_data.csv'
# training_data_dir = root_dir + '/' + 'training_data'
model_dir = root_dir + '/' + 'model'

In [6]:
import tensorflow as tf

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
print(gpu_info)

Fri Aug  5 20:33:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.91.03    Driver Version: 460.91.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:01:00.0 Off |                  N/A |
| 35%   42C    P2    89W / 260W |  10486MiB / 11019MiB |      6%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [7]:
pwd

'/home/jk21/Documents/hmd/jk_classifier'

In [8]:
# data_folder =  'physionet.org/files/circor-heart-sound/1.0.3/training_data'
train_folder =  '/home/jk21/Downloads/Data/data/murmur/train'
test_folder = '/home/jk21/Downloads/Data/data/murmur/test'


In [9]:
model_folder = 'lcnn2'

In [10]:
model_folder

'lcnn2'

In [11]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import LearningRateScheduler,ModelCheckpoint

In [12]:
def sigmoidal_decay(e, start=0, end=100, lr_start=1e-3, lr_end=1e-5):
    if e < start:
        return lr_start
    elif e > end:
        return lr_end

    middle = (start + end) / 2
    s = lambda x: 1 / (1 + np.exp(-x))

    return s(13 * (-e + middle) / np.abs(end - start)) * np.abs(lr_start - lr_end) + lr_end

In [13]:
patient_files_trn = find_patient_files(train_folder)
patient_files_test = find_patient_files(test_folder)

In [14]:
params_feature = {'samp_sec': 20,
                  #### melspec, stft 피쳐 옵션들  
                  'pre_emphasis': 0,
                  'hop_length': 256,
                  'win_length':512,
                  'n_mels': 120,
                  #### cqt 피쳐 옵션들  
                  'filter_scale': 1,
                  'n_bins': 80,
                  'fmin': 10,
                  'maxlen1': 120000,
                  'min_dist':500,
                  'max_interval_len' : 115,
                  'trim' :1
}

In [16]:
features_trn,mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape,wav2_input_shape = get_features_3lb_all(train_folder, patient_files_trn, **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_raw=False,use_interval=True,use_wav2=True)

100%|███████████████████████████████████████| 751/751 [1:02:24<00:00,  4.99s/it]


melspec:  120 313
cqt:  1 1
stft:  1 1
interval:  115 1
wav2:  374 32


In [23]:
features_test,mel_input_shape, cqt_input_shape,stft_input_shape,interval_input_shape,wav2_input_shape= get_features_3lb_all(test_folder, patient_files_test, **params_feature,use_mel = True, use_cqt = False, use_stft = False,use_raw=False,use_interval=True,use_wav2=True)

100%|█████████████████████████████████████████| 191/191 [20:34<00:00,  6.46s/it]


melspec:  120 313
cqt:  1 1
stft:  1 1
interval:  115 1
wav2:  374 32


In [17]:
with open('/home/jk21/Downloads/Data/features_trn_wav2.pickle', 'wb') as f:
    pickle.dump(features_trn, f, pickle.HIGHEST_PROTOCOL)

In [24]:
with open('/home/jk21/Downloads/Data/features_test_wav2.pickle', 'wb') as f:
    pickle.dump(features_test, f, pickle.HIGHEST_PROTOCOL)

In [15]:
with open('/home/jk21/Downloads/Data/features_trn_wav2.pickle', 'rb') as f:
    features_trn = pickle.load(f)

In [16]:
with open('/home/jk21/Downloads/Data/features_test_wav2.pickle', 'rb') as f:
    features_test = pickle.load(f)

In [17]:
mel_input_shape = features_trn['mel1'].shape[1:]

In [18]:
mel_input_shape

(120, 313, 1)

In [19]:
cqt_input_shape = features_trn['cqt1'].shape[1:]

In [20]:
cqt_input_shape

(1, 1, 1)

In [21]:
stft_input_shape = features_trn['stft1'].shape[1:]

In [22]:
stft_input_shape

(1, 1, 1)

In [23]:
interval_input_shape= features_trn['interval'].shape[1:]

In [24]:
interval_input_shape

(115, 1)

In [25]:
wav2_input_shape= features_trn['wav2'].shape[1:]

In [26]:
wav2_input_shape

(374, 32)

In [27]:
features_test['raw1']

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],

In [28]:
for i in range(len(features_trn['interval'])):
    features_trn['interval'][i]=np.array(features_trn['interval'][i])

In [29]:
for i in range(len(features_trn['mel1'])):
    features_trn['mel1'][i]=np.array(features_trn['mel1'][i])

In [30]:
model1 = get_LCNN_o_4_dr(mel_input_shape, cqt_input_shape,stft_input_shape, interval_input_shape, wav2_input_shape,use_mel=True, use_stft = False,use_cqt=False,ext = True,ord1=False)

In [31]:

model2 = get_LCNN_o_4_dr(mel_input_shape, cqt_input_shape,stft_input_shape, interval_input_shape, wav2_input_shape,use_mel=True, use_stft = False,use_cqt=False,ext = True,ord1=True)


In [32]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mel (InputLayer)               [(None, 120, 313, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 120, 313, 32  832         ['mel[0][0]']                    
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 120, 313, 32  832         ['mel[0][0]']                    
                                )                                                             

 batch_normalization_4 (BatchNo  (None, 372, 9)      1488        ['conv1d_4[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 maximum_6 (Maximum)            (None, 15, 40, 32)   0           ['conv2d_12[0][0]',              
                                                                  'conv2d_13[0][0]']              
                                                                                                  
 conv1d_1 (Conv1D)              (None, 111, 9)       252         ['batch_normalization[0][0]']    
                                                                                                  
 conv1d_5 (Conv1D)              (None, 370, 9)       252         ['batch_normalization_4[0][0]']  
                                                                                                  
 batch_nor

                                                                 ]                                
                                                                                                  
 dense_5 (Dense)                (None, 3)            24          ['global_average_pooling1d_1[0][0
                                                                 ]']                              
                                                                                                  
 global_average_pooling2d (Glob  (None, 32)          0           ['max_pooling2d_3[0][0]']        
 alAveragePooling2D)                                                                              
                                                                                                  
 concatenate (Concatenate)      (None, 14)           0           ['dense[0][0]',                  
                                                                  'dense_1[0][0]',                
          

In [33]:
model2.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 mel (InputLayer)               [(None, 120, 313, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_18 (Conv2D)             (None, 120, 313, 32  832         ['mel[0][0]']                    
                                )                                                                 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 120, 313, 32  832         ['mel[0][0]']                    
                                )                                                           

 batch_normalization_18 (BatchN  (None, 372, 9)      1488        ['conv1d_12[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 maximum_15 (Maximum)           (None, 15, 40, 32)   0           ['conv2d_30[0][0]',              
                                                                  'conv2d_31[0][0]']              
                                                                                                  
 conv1d_9 (Conv1D)              (None, 111, 9)       252         ['batch_normalization_14[0][0]'] 
                                                                                                  
 conv1d_13 (Conv1D)             (None, 370, 9)       252         ['batch_normalization_18[0][0]'] 
                                                                                                  
 batch_nor

                                                                 ]']                              
                                                                                                  
 dense_13 (Dense)               (None, 3)            24          ['global_average_pooling1d_3[0][0
                                                                 ]']                              
                                                                                                  
 global_average_pooling2d_1 (Gl  (None, 32)          0           ['max_pooling2d_7[0][0]']        
 obalAveragePooling2D)                                                                            
                                                                                                  
 concatenate_2 (Concatenate)    (None, 14)           0           ['dense_8[0][0]',                
                                                                  'dense_9[0][0]',                
          

In [34]:
n_epoch = 50
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))


batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
          'lowpass': [.5, [11,12,13,14,15,16,17,18]],
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
#           'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
            features_trn['interval'],features_trn['wav2'],features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
           features_trn['mm_labels'],  ## our Y
                        **params)()


ValidDGen_1 = Generator0([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['interval'],
                              features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
                             features_test['mm_labels'],  ## our Y
                        **params)()

class_weight = {0: 3, 1: 1., 2: 1} 

 
model1.fit(TrainDGen_1, 
           validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['interval'],
                              features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
                             features_test['mm_labels']),
           callbacks=[lr],
           steps_per_epoch=np.ceil(len(features_trn['mm_labels'])/batch_size), 
           validation_steps=np.ceil(len(features_test['mm_labels'])/batch_size)
           ,epochs = n_epoch)



Epoch 1/50
40/40 [==============================] - 7s 100ms/step - loss: 4.9376 - accuracy: 0.3820 - auc: 0.5800 - val_loss: 15.6309 - val_accuracy: 0.7448 - val_auc: 0.8083 - lr: 9.9851e-04
Epoch 2/50
40/40 [==============================] - 3s 76ms/step - loss: 1.2022 - accuracy: 0.6375 - auc: 0.8198 - val_loss: 1.5935 - val_accuracy: 0.7448 - val_auc: 0.8258 - lr: 9.9807e-04
Epoch 3/50
40/40 [==============================] - 3s 77ms/step - loss: 0.9376 - accuracy: 0.6750 - auc: 0.8337 - val_loss: 1.3970 - val_accuracy: 0.7448 - val_auc: 0.8375 - lr: 9.9750e-04
Epoch 4/50
40/40 [==============================] - 3s 77ms/step - loss: 0.7825 - accuracy: 0.7051 - auc: 0.8462 - val_loss: 0.7047 - val_accuracy: 0.7448 - val_auc: 0.8642 - lr: 9.9676e-04
Epoch 5/50
40/40 [==============================] - 3s 77ms/step - loss: 0.7263 - accuracy: 0.7078 - auc: 0.8549 - val_loss: 0.6211 - val_accuracy: 0.7655 - val_auc: 0.8741 - lr: 9.9581e-04
Epoch 6/50
40/40 [==============================

Epoch 44/50
40/40 [==============================] - 3s 77ms/step - loss: 0.5485 - accuracy: 0.7766 - auc: 0.8910 - val_loss: 0.4521 - val_accuracy: 0.8336 - val_auc: 0.9195 - lr: 1.9102e-05
Epoch 45/50
40/40 [==============================] - 3s 78ms/step - loss: 0.5280 - accuracy: 0.7898 - auc: 0.8972 - val_loss: 0.4561 - val_accuracy: 0.8336 - val_auc: 0.9195 - lr: 1.7033e-05
Epoch 46/50
40/40 [==============================] - 3s 78ms/step - loss: 0.5302 - accuracy: 0.7852 - auc: 0.8966 - val_loss: 0.4475 - val_accuracy: 0.8368 - val_auc: 0.9214 - lr: 1.5431e-05
Epoch 47/50
40/40 [==============================] - 3s 78ms/step - loss: 0.5339 - accuracy: 0.7715 - auc: 0.8987 - val_loss: 0.4496 - val_accuracy: 0.8415 - val_auc: 0.9222 - lr: 1.4193e-05
Epoch 48/50
40/40 [==============================] - 3s 78ms/step - loss: 0.5341 - accuracy: 0.7711 - auc: 0.8969 - val_loss: 0.4466 - val_accuracy: 0.8415 - val_auc: 0.9220 - lr: 1.3236e-05
Epoch 49/50
40/40 [==========================

In [35]:
n_epoch = 50
lr = LearningRateScheduler(lambda e: sigmoidal_decay(e, end=n_epoch))


batch_size = 64
params = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': True,
          'beta_param': 0.7,
          'mixup': True,
         'lowpass': [.5, [11,12,13,14,15,16,17,18]],
          'highpass': [.5, [78,79,80,81,82,83,84,85]],
#           'ranfilter2' : [3, [18,19,20,21,22,23]]
#           'dropblock' : [30, 100]
          #'device' : device
}

params_no_shuffle = {'batch_size': batch_size,
#          'input_shape': (100, 313, 1),
          'shuffle': False,
          'beta_param': 0.7,
          'mixup': False
          #'device': device
}

TrainDGen_1 = Generator0([features_trn['age'],features_trn['sex'], features_trn['hw'], features_trn['preg'], features_trn['loc'],
            features_trn['interval'],features_trn['wav2'],features_trn['mel1'],features_trn['cqt1'],features_trn['stft1']], 
           features_trn['out_labels'],  ## our Y
                        **params)()

#ValDGen_1 = Generator0([features_test[0]['age'],features_test[0]['sex'], features_test[0]['hw'], features_test[0]['preg'], features_test[0]['loc'], 
#           features_test[0]['mel1'],features_test[0]['cqt1'],features_test[0]['stft1']], 
#                        features_test[2],  ## our Y
#                        **params_no_shuffle)()
class_weight = {0: 3, 1: 1., 2: 1} 
    
model2.fit(TrainDGen_1,
           validation_data = ([features_test['age'],features_test['sex'], features_test['hw'], 
                              features_test['preg'], features_test['loc'],features_test['interval'],
                              features_test['wav2'], features_test['mel1'],features_test['cqt1'],features_test['stft1']], 
                             features_test['out_labels']),
           callbacks=[lr],
           steps_per_epoch=np.ceil(len(features_trn['out_labels'])/batch_size), 
           validation_steps=np.ceil(len(features_test['out_labels'])/batch_size),
           epochs = n_epoch)


Epoch 1/50
40/40 [==============================] - 6s 91ms/step - loss: 1.8822 - accuracy: 0.4977 - auc: 0.4936 - val_loss: 9.2659 - val_accuracy: 0.5008 - val_auc: 0.5008 - lr: 9.9851e-04
Epoch 2/50
40/40 [==============================] - 3s 78ms/step - loss: 1.1263 - accuracy: 0.4938 - auc: 0.4958 - val_loss: 2.2393 - val_accuracy: 0.5008 - val_auc: 0.4944 - lr: 9.9807e-04
Epoch 3/50
40/40 [==============================] - 3s 78ms/step - loss: 0.9068 - accuracy: 0.4969 - auc: 0.5052 - val_loss: 1.3025 - val_accuracy: 0.5024 - val_auc: 0.4846 - lr: 9.9750e-04
Epoch 4/50
40/40 [==============================] - 3s 78ms/step - loss: 0.7781 - accuracy: 0.5164 - auc: 0.5255 - val_loss: 0.8570 - val_accuracy: 0.5151 - val_auc: 0.5032 - lr: 9.9676e-04
Epoch 5/50
40/40 [==============================] - 3s 78ms/step - loss: 0.7405 - accuracy: 0.5176 - auc: 0.5230 - val_loss: 0.7834 - val_accuracy: 0.5119 - val_auc: 0.5089 - lr: 9.9581e-04
Epoch 6/50
40/40 [==============================] 

Epoch 44/50
40/40 [==============================] - 3s 79ms/step - loss: 0.6788 - accuracy: 0.5766 - auc: 0.6402 - val_loss: 0.6428 - val_accuracy: 0.6593 - val_auc: 0.7017 - lr: 1.9102e-05
Epoch 45/50
40/40 [==============================] - 3s 78ms/step - loss: 0.6719 - accuracy: 0.5836 - auc: 0.6557 - val_loss: 0.6424 - val_accuracy: 0.6513 - val_auc: 0.6987 - lr: 1.7033e-05
Epoch 46/50
40/40 [==============================] - 3s 80ms/step - loss: 0.6754 - accuracy: 0.5816 - auc: 0.6416 - val_loss: 0.6421 - val_accuracy: 0.6545 - val_auc: 0.7010 - lr: 1.5431e-05
Epoch 47/50
40/40 [==============================] - 3s 78ms/step - loss: 0.6761 - accuracy: 0.5973 - auc: 0.6462 - val_loss: 0.6422 - val_accuracy: 0.6656 - val_auc: 0.7001 - lr: 1.4193e-05
Epoch 48/50
40/40 [==============================] - 3s 78ms/step - loss: 0.6736 - accuracy: 0.5727 - auc: 0.6412 - val_loss: 0.6412 - val_accuracy: 0.6577 - val_auc: 0.7039 - lr: 1.3236e-05
Epoch 49/50
40/40 [==========================

In [36]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 120000,
 'min_dist': 500,
 'max_interval_len': 115,
 'trim': 1}

In [37]:
params_feature['mel_shape'] = mel_input_shape
params_feature['cqt_shape'] = cqt_input_shape
params_feature['stft_shape'] = stft_input_shape
params_feature['interval_input_shape'] = interval_input_shape
params_feature['wav2_input_shape'] = wav2_input_shape

In [43]:
params_feature

{'samp_sec': 20,
 'pre_emphasis': 0,
 'hop_length': 256,
 'win_length': 512,
 'n_mels': 120,
 'filter_scale': 1,
 'n_bins': 80,
 'fmin': 10,
 'maxlen1': 120000,
 'min_dist': 500,
 'max_interval_len': 115,
 'trim': 1,
 'mel_shape': (120, 313, 1),
 'cqt_shape': (1, 1, 1),
 'stft_shape': (1, 1, 1),
 'interval_input_shape': (115, 1),
 'wav2_input_shape': (374, 32),
 'model1': 'lcnn1',
 'model2': 'lcnn2',
 'model_fnm1': 'lcnn2/lcnn1_model1.hdf5',
 'model_fnm2': 'lcnn2/lcnn2_model2.hdf5'}

In [38]:
import pickle as pk
def save_challenge_model(model_folder, model1, model2, m_name1, m_name2, param_feature) :
    os.makedirs(model_folder, exist_ok=True)
    info_fnm = os.path.join(model_folder, 'desc.pk')
    filename1 = os.path.join(model_folder, m_name1 + '_model1.hdf5')
    filename2 = os.path.join(model_folder, m_name2 + '_model2.hdf5')
    model1.save(filename1)
    model2.save(filename2)
    param_feature['model1'] = m_name1
    param_feature['model2'] = m_name2
    param_feature['model_fnm1'] = filename1
    param_feature['model_fnm2'] = filename2

    with open(info_fnm, 'wb') as f:
        pk.dump(param_feature, f, pk.HIGHEST_PROTOCOL)
    return 1
    

In [39]:
model_folder

'lcnn2'

In [40]:
save_challenge_model(model_folder, model1, model2, m_name1 = 'lcnn1', m_name2 = 'lcnn2', param_feature = params_feature)

1

In [44]:
def load_challenge_model(model_folder, verbose):
    info_fnm = os.path.join(model_folder, 'desc.pk')
    with open(info_fnm, 'rb') as f:
        info_m = pk.load(f)
#    if info_m['model'] == 'toy' :
#        model = get_toy(info_m['mel_shape'])
#    filename = os.path.join(model_folder, info_m['model'] + '_model.hdf5')
#    model.load_weights(filename)
    return info_m

In [45]:
def run_challenge_model(model, data, recordings, verbose,use_mel=True,use_stft=False,use_cqt=False,
                        maxlen=80,min_dist=500):
    
    murmur_classes = ['Present', 'Unknown', 'Absent']
    outcome_classes = ['Abnormal', 'Normal']
    
    if model['model1'] == 'lcnn1' :
        model1 = get_LCNN_o_4_dr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_input_shape'],model['wav2_input_shape'],use_mel = True, use_stft = False,use_cqt=False,ord1=False,ext=True)
    if model['model2'] == 'lcnn2' :
        model2 = get_LCNN_o_4_dr(model['mel_shape'],model['cqt_shape'],model['stft_shape'], model['interval_input_shape'], model['wav2_input_shape'],use_mel = True,use_stft = False,use_cqt=False,ord1=True,ext=True)
    model1.load_weights(model['model_fnm1'])
    model2.load_weights(model['model_fnm2'])
    
    max_interval_len = params_feature['max_interval_len']
    maxlen1 = params_feature['maxlen1']
    min_dist = params_feature['min_dist']
#    classes = model['classes']
    # Load features.
    features = get_feature_one(data, verbose = 0)

    samp_sec = model['samp_sec'] 
    pre_emphasis = model['pre_emphasis']
    hop_length = model['hop_length']
    win_length = model['win_length']
    n_mels = model['n_mels']
    filter_scale = model['filter_scale']
    n_bins = model['n_bins']
    fmin = model['fmin']
    

    features['interval'] = []
    tmp_interval=[]  
    
    features['wav2'] = []
    tmp_raw = []
    tmp=[]      
    
    from wav2vec2 import Wav2Vec2ForCTC, Wav2Vec2Config
    import tensorflow as tf
    
    config = Wav2Vec2Config()
    tmp_model = Wav2Vec2ForCTC(config)
    tmp_model.trainable=False
    
    for i in range(len(recordings)):
        data = recordings[i]/32768
        tmp_raw.append(data)
        
    tmp_pad =pad_sequences(tmp_raw, maxlen=maxlen1, dtype='float64', padding='pre', truncating='pre', value=0.0)
    tmp_pad = tmp_pad[:,np.newaxis,:]
    
    for i in tqdm.tqdm(range(len(tmp_pad))):
        tmp_feature = tmp_model(tmp_pad[i])
        tmp.append(tmp_feature)
    
    tmp=np.array(tmp, dtype=np.float32)
    new_tmp1=tmp.reshape(-1,374,32)
    features['wav2']=new_tmp1
    
    
    
    for i in range(len(recordings)) :
        datos=recordings[i]
        filtros=sio.loadmat('/home/jk21/Documents/hmd/jk_classifier/S1-S1-Phonocardiogram-Peak-Detection-Method-in-Python/Filters1')
        try:
            X=datos
            X=X[12000:-12000]
            Fs=4000
            
            Fpa20=filtros['Fpa20'];			        # High pass filter
            Fpa20=Fpa20[0];					# High pass filter
            Fpb100=filtros['Fpb100'];		        # Low-pass Filter
            Fpb100=Fpb100[0];				# Low-pass Filter
            
            Xf=FpassBand(X,Fpa20,Fpb100); 	                # Apply a passband filter
            Xf=vec_nor(Xf);			
            
                    # Derivate of the Signal
            dX=derivate(Xf);				# Derivate of the signal
            dX=vec_nor(dX);					# Vector Normalizing
                    # Square of the signal
            dy=np.square(Xf);
            dy=vec_nor(dy);
                    
            size=np.shape(Xf)				# Rank or dimension of the array
            fil=size[0];					# Number of rows

            positive=np.zeros((1,fil+1));                   # Initializating Positives Values Vector 
            positive=positive[0];                           # Getting the Vector

            points=np.zeros((1,fil));                       # Initializating the all Peak Points Vector
            points=points[0];                               # Getting the point vector

            peaks=np.zeros((1,fil));                        # Initializating the s1-s1 Peak Vector
            peaks=peaks[0];                                 # Getting the point vector

            
            for i in range(0,fil):
                if dX[i]>0:
                    positive[i]=1;
                else:
                    positive[i]=0;

            for i in range(0,fil):
                if (positive[i]==1 and positive[i+1]==0):
                    points[i]=Xf[i];
                else:
                    points[i]=0;

            indexes=peakutils.indexes(points,thres=0.5/max(points), min_dist=min_dist);
            lenght=np.shape(indexes)			# Get the length of the index vector		
            lenght=lenght[0];				# Get the value of the index vector

            for i in range(0,lenght):
                p=indexes[i];
                peaks[p]=points[p];
        
            n=np.arange(0,fil);                            # Vector to the X axes (Number of Samples)
            
            tmp_peaks = np.diff(indexes)
                    
            tmp_interval.append(tmp_peaks)
                    
           
        except:
            print(i)
            tmp_peaks = np.zeros(maxlen)
            tmp_interval.append(tmp_peaks)
                    
     
    padded =pad_sequences(tmp_interval, maxlen=max_interval_len, dtype='float64', padding='pre', truncating='pre', value=0.0)
        
    for i in range(len(padded)):
        features['interval'].append(padded[i])
        
        
    for i in range(len(features['interval'])):
        features['interval'][i] = np.array(features['interval'][i])
        
    features['interval'] = np.array(features['interval'])
    
    
    
    features['mel1'] = []
    for i in range(len(recordings)) :
        if use_mel :
            mel1 = feature_extract_melspec(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                           win_length = win_length, n_mels = n_mels)[0]
        else :
            mel1 = np.zeros( (1,1) )
            
        features['mel1'].append(mel1)
        
    M, N = features['mel1'][0].shape
    
    if use_mel :
        for i in range(len(features['mel1'])) :
            features['mel1'][i] = features['mel1'][i].reshape(M,N,1)   
    
    features['mel1'] = np.array(features['mel1'])

    
  
    features['cqt1'] = []
    for i in range(len(recordings)) :
        if use_cqt :
            mel1 = feature_extract_cqt(recordings[i], samp_sec=samp_sec, pre_emphasis = pre_emphasis, filter_scale = filter_scale, 
                                        n_bins = n_bins, fmin = fmin)[0]
        else:
            mel1 = np.zeros( (1,1,1) )
            
        features['cqt1'].append(mel1)
        
    M, N,__ = features['cqt1'][0].shape
    
    if use_cqt :
        for i in range(len(features['cqt1'])) :
            features['cqt1'][i] = features['cqt1'][i].reshape(M,N,1)   
    
    features['cqt1'] = np.array(features['cqt1'])
    
    
    features['stft1'] = []
    for i in range(len(recordings)) :
        if use_stft :
            mel1 = feature_extract_stft(recordings[i]/ 32768, samp_sec=samp_sec, pre_emphasis = pre_emphasis, hop_length=hop_length, 
                                        win_length = win_length)[0]
        else :
            mel1 = np.zeros( (1,1,1) )
        
        features['stft1'].append(mel1)
        
    M, N,__ = features['stft1'][0].shape
    if use_stft :
        for i in range(len(features['stft1'])) :
            features['stft1'][i] = features['stft1'][i].reshape(M,N,1)           
    features['stft1'] = np.array(features['stft1'])
    
#     return features       

    #    print(features)
    # Impute missing data.
    res1 = model1.predict([features['age'], features['sex'], features['hw'], features['preg'], 
                           features['loc'], features['interval'],features['wav2'],features['mel1'], features['stft1'], features['cqt1']])
    res2 = model2.predict([features['age'], features['sex'], features['hw'], features['preg'], 
                           features['loc'], features['interval'],features['wav2'],features['mel1'], features['stft1'], features['cqt1']])

    # Get classifier probabilities.
    idx1 = res1.argmax(axis=0)[0]
    murmur_probabilities = res1[idx1,]  ## mumur 확률 최대화 되는 애 뽑기
    outcome_probabilities = res2.mean(axis = 0) ##  outcome 은 그냥 평균으로 뽑기
#    idx = np.argmax(prob1)

    ## 이부분도 생각 필요.. rule 을 cost를 maximize 하는 기준으로 threshold 탐색 필요할지도..
    # Choose label with highest probability.
    murmur_labels = np.zeros(len(murmur_classes), dtype=np.int_)
    idx = np.argmax(murmur_probabilities)
    murmur_labels[idx] = 1
    outcome_labels = np.zeros(len(outcome_classes), dtype=np.int_)
    idx = np.argmax(outcome_probabilities)
    outcome_labels[idx] = 1
    
    # Concatenate classes, labels, and probabilities.
    classes = murmur_classes + outcome_classes
    labels = np.concatenate((murmur_labels, outcome_labels))
    probabilities = np.concatenate((murmur_probabilities, outcome_probabilities))
    
    return classes, labels, probabilities

In [46]:
model_folder

'lcnn2'

In [47]:
data_folder = test_folder

In [48]:
data_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [49]:
output_folder ='/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [50]:
allow_failures=True

In [51]:
verbose=1

In [52]:
model = load_challenge_model(model_folder, verbose)

In [53]:
patient_files = find_patient_files(data_folder)

In [54]:
num_patient_files = len(patient_files)

In [55]:
patient_data = load_patient_data(patient_files[0])

In [56]:
patient_data

'33151 4 4000\nAV 33151_AV.hea 33151_AV.wav 33151_AV.tsv\nPV 33151_PV.hea 33151_PV.wav 33151_PV.tsv\nTV 33151_TV.hea 33151_TV.wav 33151_TV.tsv\nMV 33151_MV.hea 33151_MV.wav 33151_MV.tsv\n#Age: Child\n#Sex: Female\n#Height: 141.0\n#Weight: 30.9\n#Pregnancy status: False\n#Murmur: Present\n#Murmur locations: MV+TV\n#Most audible location: TV\n#Systolic murmur timing: Holosystolic\n#Systolic murmur shape: Plateau\n#Systolic murmur grading: I/VI\n#Systolic murmur pitch: Low\n#Systolic murmur quality: Harsh\n#Diastolic murmur timing: nan\n#Diastolic murmur shape: nan\n#Diastolic murmur grading: nan\n#Diastolic murmur pitch: nan\n#Diastolic murmur quality: nan\n#Outcome: Abnormal\n#Campaign: CC2015\n#Additional ID: nan\n'

In [57]:
recordings = load_recordings(data_folder, patient_data)

In [58]:
classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose)

100%|█████████████████████████████████████████████| 4/4 [00:00<00:00,  8.27it/s]


In [59]:
classes

['Present', 'Unknown', 'Absent', 'Abnormal', 'Normal']

In [60]:
# Run model.
def run_model(model_folder, data_folder, output_folder, allow_failures, verbose=1):
    # Load models.
    if verbose >= 1:
        print('Loading Challenge model...')

    model = load_challenge_model(model_folder, verbose) ### Teams: Implement this function!!!

    # Find the patient data files.
    patient_files = find_patient_files(data_folder)
    num_patient_files = len(patient_files)

    if num_patient_files==0:
        raise Exception('No data was provided.')

    # Create a folder for the Challenge outputs if it does not already exist.
    os.makedirs(output_folder, exist_ok=True)

    # Run the team's model on the Challenge data.
    if verbose >= 1:
        print('Running model on Challenge data...')

    # Iterate over the patient files.
    for i in range(num_patient_files):
        if verbose >= 2:
            print('    {}/{}...'.format(i+1, num_patient_files))

        patient_data = load_patient_data(patient_files[i])
        recordings = load_recordings(data_folder, patient_data)

        # Allow or disallow the model to fail on parts of the data; helpful for debugging.
        try:
            classes, labels, probabilities = run_challenge_model(model, patient_data, recordings, verbose) ### Teams: Implement this function!!!
        except:
            if allow_failures:
                if verbose >= 2:
                    print('... failed.')
                classes, labels, probabilities = list(), list(), list()
            else:
                raise

        # Save Challenge outputs.
        head, tail = os.path.split(patient_files[i])
        root, extension = os.path.splitext(tail)
        output_file = os.path.join(output_folder, root + '.csv')
        patient_id = get_patient_id(patient_data)
        save_challenge_outputs(output_file, patient_id, classes, labels, probabilities)

    if verbose >= 1:
        print('Done.')

In [61]:
test_folder

'/home/jk21/Downloads/Data/data/murmur/test'

In [62]:
output_folder = '/home/jk21/Documents/hmd/jk_classifier/out_lcnn2'

In [63]:
run_model(model_folder, test_folder, output_folder, allow_failures = True, verbose = 2)

Loading Challenge model...
Running model on Challenge data...
    1/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 11.52it/s]


    2/191...


100%|█████████████████████████████████████████████| 2/2 [00:00<00:00,  9.10it/s]


    3/191...


...

    190/191...


100%|█████████████████████████████████████████████| 4/4 [00:00<00:00, 12.16it/s]


    191/191...


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 12.01it/s]


Done.


In [64]:
from evaluate_model import *

In [65]:
murmur_scores, outcome_scores = evaluate_model(test_folder, output_folder)
classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
    + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string

if len(sys.argv) == 3:
    print(output_string)
elif len(sys.argv) == 4:
    with open(sys.argv[3], 'w') as f:
        f.write(output_string)

#Murmur scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.469,0.348,0.127,0.215,0.520,14496.256

#Outcome scores
AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost
0.578,0.597,0.363,0.524,0.844,14706.772

#Murmur scores (per class)
Classes,Present,Unknown,Absent
AUROC,0.489,0.404,0.515
AUPRC,0.195,0.113,0.735
F-measure,0.338,0.000,0.042
Accuracy,1.000,0.000,0.022

#Outcome scores (per class)
Classes,Abnormal,Normal
AUROC,0.578,0.578
AUPRC,0.627,0.567
F-measure,0.683,0.042
Accuracy,1.000,0.022



In [ ]:
label_folder = test_folder
murmur_classes = ['Present', 'Unknown', 'Absent']
outcome_classes = ['Abnormal', 'Normal']

# Load and parse label and model output files.
label_files, output_files = find_challenge_files(label_folder, output_folder)
murmur_labels = load_murmurs(label_files, murmur_classes)
murmur_binary_outputs, murmur_scalar_outputs = load_classifier_outputs(output_files, murmur_classes)
outcome_labels = load_outcomes(label_files, outcome_classes)
outcome_binary_outputs, outcome_scalar_outputs = load_classifier_outputs(output_files, outcome_classes)


print(np.mean(murmur_scalar_outputs[:,0]))
print(np.mean(murmur_scalar_outputs[:,2]))
print(np.mean(outcome_scalar_outputs[:,0]))
print(np.mean(outcome_scalar_outputs[:,1]))

In [ ]:
for th1 in [0.01, 0.05, 0.1, 0.15,0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8] :
    murmur_binary_outputs[:,0] = murmur_scalar_outputs[:,0] > th1
    murmur_binary_outputs[:,2] = murmur_scalar_outputs[:,2] > 1 - th1
    outcome_binary_outputs[:,0] = outcome_scalar_outputs[:,0] > th1
    outcome_binary_outputs[:,1] = outcome_scalar_outputs[:,1] > 1 - th1
    # For each patient, set the 'Present' or 'Abnormal' class to positive if no class is positive or if multiple classes are positive.
    murmur_labels = enforce_positives(murmur_labels, murmur_classes, 'Present')
    murmur_binary_outputs = enforce_positives(murmur_binary_outputs, murmur_classes, 'Present')
    outcome_labels = enforce_positives(outcome_labels, outcome_classes, 'Abnormal')
    outcome_binary_outputs = enforce_positives(outcome_binary_outputs, outcome_classes, 'Abnormal')
    # Evaluate the murmur model by comparing the labels and model outputs.
    murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes = compute_auc(murmur_labels, murmur_scalar_outputs)
    murmur_f_measure, murmur_f_measure_classes = compute_f_measure(murmur_labels, murmur_binary_outputs)
    murmur_accuracy, murmur_accuracy_classes = compute_accuracy(murmur_labels, murmur_binary_outputs)
    murmur_weighted_accuracy = compute_weighted_accuracy(murmur_labels, murmur_binary_outputs, murmur_classes) # This is the murmur scoring metric.
    murmur_cost = compute_cost(outcome_labels, murmur_binary_outputs, outcome_classes, murmur_classes) # Use *outcomes* to score *murmurs* for the Challenge cost metric, but this is not the actual murmur scoring metric.
    murmur_scores = (murmur_classes, murmur_auroc, murmur_auprc, murmur_auroc_classes, murmur_auprc_classes, \
                 murmur_f_measure, murmur_f_measure_classes, murmur_accuracy, murmur_accuracy_classes, murmur_weighted_accuracy, murmur_cost)

    # Evaluate the outcome model by comparing the labels and model outputs.
    outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes = compute_auc(outcome_labels, outcome_scalar_outputs)
    outcome_f_measure, outcome_f_measure_classes = compute_f_measure(outcome_labels, outcome_binary_outputs)
    outcome_accuracy, outcome_accuracy_classes = compute_accuracy(outcome_labels, outcome_binary_outputs)
    outcome_weighted_accuracy = compute_weighted_accuracy(outcome_labels, outcome_binary_outputs, outcome_classes)
    outcome_cost = compute_cost(outcome_labels, outcome_binary_outputs, outcome_classes, outcome_classes) # This is the clinical outcomes scoring metric.
    outcome_scores = (outcome_classes, outcome_auroc, outcome_auprc, outcome_auroc_classes, outcome_auprc_classes, \
                  outcome_f_measure, outcome_f_measure_classes, outcome_accuracy, outcome_accuracy_classes, outcome_weighted_accuracy, outcome_cost)



    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = murmur_scores
    murmur_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    murmur_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    classes, auroc, auprc, auroc_classes, auprc_classes, f_measure, f_measure_classes, accuracy, accuracy_classes, weighted_accuracy, cost = outcome_scores
    outcome_output_string = 'AUROC,AUPRC,F-measure,Accuracy,Weighted Accuracy,Cost\n{:.3f},{:.3f},{:.3f},{:.3f},{:.3f},{:.3f}\n'.format(auroc, auprc, f_measure, accuracy, weighted_accuracy, cost)
    outcome_class_output_string = 'Classes,{}\nAUROC,{}\nAUPRC,{}\nF-measure,{}\nAccuracy,{}\n'.format(
    ','.join(classes),
    ','.join('{:.3f}'.format(x) for x in auroc_classes),
    ','.join('{:.3f}'.format(x) for x in auprc_classes),
    ','.join('{:.3f}'.format(x) for x in f_measure_classes),
    ','.join('{:.3f}'.format(x) for x in accuracy_classes))

    output_string = '#Murmur scores\n' + murmur_output_string + '\n#Outcome scores\n' + outcome_output_string \
                + '\n#Murmur scores (per class)\n' + murmur_class_output_string + '\n#Outcome scores (per class)\n' + outcome_class_output_string
    print("threshold: ", th1)
    print(output_string)
    print("-------------")